In [ ]:
import torch
import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from seper.calculate import gen_answers_batch, calculate_uncertainty_soft_batch, create_collate_fn
from seper.models.huggingface_models import HuggingfaceModel
from seper.uncertainty_measures.semantic_entropy import EntailmentDeberta
from seper.calculate import process_item_for_seper


/opt/conda/envs/searchr1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/searchr1/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/opt/conda/envs/searchr1/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True wa

KeyboardInterrupt: 

#### Load Model
Load the generation model and entailment judge model.

In [ ]:
model_path = 'Qwen/Qwen3-8B'
num_generations = 10 # 10 is good for most cases
sub_batch_size = 10
temperature = 1.0
max_new_tokens = 128
max_context_words = 4096
computation_chunk_size = 8 # adjust to balance speeds and gpu memory cost
prompt_type = 'default'
device = 'cuda:1'

# Build generator
generator = HuggingfaceModel(
    model_path,
    stop_sequences='default',
    max_new_tokens=max_new_tokens,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2",
    device=device,
)
generator.model.eval()

# Build entailment model
entailment_model = EntailmentDeberta(device=device)
entailment_model.model.eval()

`torch_dtype` is deprecated! Use `dtype` instead!
Fetching 5 files:   0%|          | 0/5 [02:22<?, ?it/s]


#### Step 1: Sampling
Generate for `num_generations` times and collect the results and probabilities.

In [3]:
example={
    'question': 'Is Elon Musk older than Sam Altman?',
    'context': 'Elon Musk was born on June 28, 1971. Sam Altman was born on April 22, 1985.',
    'answers': ['Yes'], # this is the provided ground-truth answers
}
result = gen_answers_batch(example, 
                           generator, 
                           temperature, 
                           num_generations, 
                           sub_batch_size, 
                           max_new_tokens, 
                           prompt_type, 
                           device,
                           max_context_words)

# Baseline example with empty context
example_baseline = example.copy()
example_baseline['context'] = ''

result_baseline = gen_answers_batch(
    example_baseline,
    generator,
    temperature,
    num_generations,
    sub_batch_size,
    max_new_tokens,
    prompt_type,
    device,
    max_context_words
)

In [4]:
result

{'question': 'Is Elon Musk older than Sam Altman?',
 'context': 'Elon Musk was born on June 28, 1971. Sam Altman was born on April 22, 1985.',
 'answers': ['Yes'],
 'responses': [('Yes. Elon Musk was born 9 years before Sam Altman.',
   [-0.011683372780680656,
    -0.22827866673469543,
    -1.8856996297836304,
    -1.0728830375228426e-06,
    -1.9192511899746023e-05,
    -3.814689989667386e-06,
    -1.063927412033081,
    -1.3947389561508317e-05,
    -1.4852174520492554,
    -0.06590946763753891,
    -0.000164018536452204,
    -0.011408685706555843,
    -0.00011860620725201443,
    -4.851700214203447e-05,
    -3.814689989667386e-06,
    -0.0003120412293355912]),
  ('Yes.', [-0.011683372780680656, -0.22827866673469543]),
  ('Yes. Elon Musk was born in 1971, while Sam Altman was born in 1985, making Elon Musk older than Sam Altman.',
   [-0.011683372780680656,
    -0.22827866673469543,
    -1.8856996297836304,
    -1.0728830375228426e-06,
    -1.9192511899746023e-05,
    -3.8146899896673

#### Step 2: Calculate ΔSePer
Aggregate semantic-equivalent answers and compute belief shift on gt answer.

In [5]:
keys = ['question', 'response_text', 'answers', 'likelihood', 'context_label', 'log_liks_agg', 'context']
seper_collate_fn = create_collate_fn(keys)

# calculate seper
with torch.no_grad():
    # Convert for SEPER
    r = process_item_for_seper(result)
    rb = process_item_for_seper(result_baseline)
    seper_input = seper_collate_fn([r, rb])
    seper, seper_baseline = calculate_uncertainty_soft_batch(
        seper_input, entailment_model, computation_chunk_size
    )
    d_seper = seper - seper_baseline

### Extented Results: Fine-grained Retrieval Utility Evaluation

In [6]:
example_baseline={
    'question': 'Is Elon Musk older than Sam Altman?',
    'context': '',
    'answers': ['Yes'], # this is the provided ground-truth answers
}

example_piece1 = example_baseline.copy()
example_piece1['context'] = 'Elon Musk was born on June 28, 1971.'
example_piece2 = example_baseline.copy()
example_piece2['context'] = 'Sam Altman was born on April 22, 1985.'

result_piece1 = gen_answers_batch(example_piece1, generator, temperature, num_generations, sub_batch_size, max_new_tokens, prompt_type, device, max_context_words)
result_piece2 = gen_answers_batch(example_piece2, generator, temperature, num_generations, sub_batch_size, max_new_tokens, prompt_type, device, max_context_words)


In [7]:
# calculate seper
with torch.no_grad():
    # Convert for SEPER
    r_1 = process_item_for_seper(result_piece1)
    r_2 = process_item_for_seper(result_piece2)
    seper_input = seper_collate_fn([r_1, r_2])
    seper_piece1, seper_piece2 = calculate_uncertainty_soft_batch(
        seper_input, entailment_model, computation_chunk_size
    )
    d_seper_piece1 = seper_piece1 - seper_baseline
    d_seper_piece2 = seper_piece2 - seper_baseline

print(
    f'(Baseline) SePer without retrieved information: {seper_baseline}\n', 
    f'SePer with two pieces of information: {seper}, ΔSePer: {d_seper}\n', 
    f'SePer with one piece of information: {seper_piece1}, ΔSePer: {d_seper_piece1}\n', 
    f'SePer with another piece of information: {seper_piece2}, ΔSePer: {d_seper_piece2}')

(Baseline) SePer without retrieved information: 0.5311136874747243
 SePer with two pieces of information: 0.9920210827879619, ΔSePer: 0.46090739531323754
 SePer with one piece of information: 0.6686031010404139, ΔSePer: 0.13748941356568956
 SePer with another piece of information: 0.8733925752846344, ΔSePer: 0.3422788878099101
